# Logistic Regression - to predict Titles

In [1]:
#load the data
import datetime
import pandas as pd
import numpy as np
import cPickle as pickle
import patsy
import unidecode

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split

SyntaxError: invalid syntax (<ipython-input-1-965ff78aa097>, line 9)

# Part 1: Load the clean data

In [17]:
# loading data from the pickled data frame

with open('master_total_df.p','rb') as f:
    master_total_df = pickle.load(f)

In [ ]:
def uncode(x):
    try:
        return unidecode.unidecode(x.decode('utf-8')).replace('\n',' -').lower()

    except:
        try:
            return unidecode.unidecode(x).replace('\n',' -').lower()
        except:
            print x
            return x 

In [32]:
master_total_df.city = master_total_df.city.map(lambda x :'other' if type(x) == dict else x)
master_total_df.city.fillna('other', inplace=True)
master_total_df.state.fillna('other', inplace=True)
master_total_df.company = master_total_df.company.map(uncode) 


print master_total_df.info()
master_total_df.head(2)


nan
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22707 entries, 0 to 22706
Data columns (total 16 columns):
index              22707 non-null int64
company            22706 non-null object
jobdesc            22707 non-null object
city               22707 non-null object
state              22707 non-null object
title              22707 non-null object
sourcesite         22707 non-null object
views              22707 non-null int64
days_posted        8038 non-null object
post_start_date    8038 non-null object
link               22707 non-null object
base_title         22707 non-null object
parsed_title       22707 non-null object
parsed_title_i     22707 non-null object
expanded_title     21261 non-null object
prefix_title       21261 non-null object
dtypes: int64(2), object(14)
memory usage: 2.8+ MB
None


/Users/tlee010/anaconda/envs/dsi/lib/python2.7/site-packages/unidecode/__init__.py:46: RuntimeWarning: Argument <type 'float'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  _warn_if_not_unicode(string)


,index,company,jobdesc,city,state,title,sourcesite,views,days_posted,post_start_date,link,base_title,parsed_title,parsed_title_i,expanded_title,prefix_title
0,0,advocate health care,"\nAs part of Advocate Health Care, Advocate Ch...",Oak Lawn,IL,Clinical Practice Specialist - 4 Hope,ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=6244f7f3a4861...,specialist,"[clinical, practice, specialist, , , 4, hope]",[2],practice specialist,practice
1,1,university of washington medical center,\nThe University of Washington (UW) is proud t...,Seattle,WA,WEB DEVELOPER,ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=3d6a9f18f5301...,developer,"[web, developer]",[1],web developer,web


In [33]:
alltitles = master_total_df.expanded_title.value_counts()
print alltitles[alltitles>100].sum()
limittitles = alltitles[alltitles>100].index
limittitles

12518


Index([u'business analyst', u'data analyst', u'software engineer',
       u'data scientist', u'data engineer', u'analyst', u'systems analyst',
       u'development engineer', u'senior consultant', u'marketing specialist',
       u'director', u'product manager', u'manager', u'senior analyst',
       u'intelligence analyst', u'research scientist', u'marketing analyst',
       u'data architect', u'operations analyst', u'project manager',
       u'solution architect', u'product analyst', u'financial analyst',
       u'senior associate', u'learning engineer', u'research analyst'],
      dtype='object')

In [34]:
limited_postings = master_total_df.loc[master_total_df['expanded_title'].isin(limittitles)].copy()
limited_postings.reset_index(inplace=True)
limited_postings.head(3)

,level_0,index,company,jobdesc,city,state,title,sourcesite,views,days_posted,post_start_date,link,base_title,parsed_title,parsed_title_i,expanded_title,prefix_title
0,2,2,macy's,"\nJob Overview:\n\n\nThe Analyst, Marketing An...",New York,NY,"Analyst, Marketing Analytics",ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=d94a7d13dfbb0...,analyst,"[analyst, , marketing, analytics]",[0],analyst,analyst
1,3,3,skyport systems,\nSkyport Systems is developing groundbreaking...,Mountain View,CA,Data Engineer,ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=c97751deb9e89...,engineer,"[data, engineer]",[1],data engineer,data
2,7,7,"iqlogg, inc","\nIQlogg, Inc. is an equal opportunity employe...",Dallas,TX,Health Care Junior Business Analyst,ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=70284c7a9286e...,analyst,"[health, care, junior, business, analyst]",[4],business analyst,business


In [150]:
limited_postings.expanded_title.value_counts()

business analyst        4029
data analyst            1530
software engineer       1285
data scientist          1139
data engineer           1085
analyst                  310
systems analyst          294
development engineer     266
senior consultant        249
marketing specialist     188
director                 183
product manager          178
manager                  170
senior analyst           155
intelligence analyst     146
marketing analyst        139
research scientist       139
data architect           132
operations analyst       124
project manager          121
solution architect       118
product analyst          115
financial analyst        111
senior associate         105
learning engineer        105
research analyst         102
Name: expanded_title, dtype: int64

## Features

The features that will be used to determine the "expanded" title are:
- city
- company
- state
- jobdesc
- jobdesc length (in characters)

First to clean up the data, the title must be taken out of the text, otherwise that will be the obvious strong predictor

In [35]:
def removeTitles(y):
    y = y.lower()
    for z in limittitles:
        y = y.replace(z,' ')
    return y

limited_postings['jobdesc'] = limited_postings['jobdesc'].map(removeTitles)
limited_postings['jobdesc'] = limited_postings['jobdesc'].map(lambda x : x.replace('\n',' '))
limited_postings['desc_len'] = limited_postings['jobdesc'].map(lambda x: len(x))


        
limited_postings['cleandesc'] = limited_postings['jobdesc'].map(uncode) 

In [36]:
limited_postings = limited_postings[['expanded_title','company','city','state','jobdesc','desc_len','cleandesc']]
limited_postings.head(2)

,expanded_title,company,city,state,jobdesc,desc_len,cleandesc
0,analyst,macy's,New York,NY,"job overview: the , marketing analytics is...",3190,"job overview: the , marketing analytics is..."
1,data engineer,skyport systems,Mountain View,CA,"skyport systems is developing groundbreaking,...",1426,"skyport systems is developing groundbreaking,..."


In [37]:
limited_postings.shape

(12518, 7)

# Word Extraction from Job Descriptions

In [24]:
text = limited_postings['cleandesc']

In [25]:
cvec = CountVectorizer(stop_words='english', lowercase=True,ngram_range=(1,1))
start_time = datetime.datetime.now()
cvec.fit(limited_postings['cleandesc'])
print datetime.datetime.now() - start_time

cdf  = pd.DataFrame(cvec.transform(text).todense(),
             columns=cvec.get_feature_names())
print datetime.datetime.now() - start_time
print cdf.shape
cdf.head(4)

0:00:06.634604
0:00:14.935094
(12518, 51258)


,00,000,00000796,00001,00001f99,00001fbh,00002581,00002581job,00002581the,000038,...,zuri,zurich,zurichrisk,zurifurniture,zurn,zuum,zycus,zyme,zymergen,zynga
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Only retain the top 10,000 words - will merge later after patsy

In [26]:
summary = cdf.sum().sort_values(ascending = False)
keep_cols = summary[:10000].index
cdf_to_merge = cdf[keep_cols]
cdf_to_merge.columns = ['nlp_'+x for x in cdf_to_merge.columns]

## Prepare y, X with multiclass tools and patsy; then add the nlp analysis fields

Patsy is limited to roughly 400 fields before pooping out 

In [183]:
formula = 'expanded_title ~ company + city + state + desc_len-1'
print formula
y, X = patsy.dmatrices(formula, limited_postings, return_type='dataframe')
# will ignore the y value for the matrix below

X = X.merge(cdf_to_merge, how = 'left', left_index=True,right_index=True)
print X.shape


expanded_title ~ company + city + state + desc_len-1
(12518, 15748)


In [184]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform([[x] for x in limited_postings['expanded_title']])
print mlb.classes_
y
print y.shape

['analyst' 'business analyst' 'data analyst' 'data architect'
 'data engineer' 'data scientist' 'development engineer' 'director'
 'financial analyst' 'intelligence analyst' 'learning engineer' 'manager'
 'marketing analyst' 'marketing specialist' 'operations analyst'
 'product analyst' 'product manager' 'project manager' 'research analyst'
 'research scientist' 'senior analyst' 'senior associate'
 'senior consultant' 'software engineer' 'solution architect'
 'systems analyst']
(12518, 26)


# Setup train test split

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(10014, 15748) (2504, 15748) (10014, 26) (2504, 26)


## Setup multi-class logistic regression as a baseline

In [210]:
def multiGres(model, X_train, y_train,X_test,y_test):
    start = datetime.datetime.now()
    OVRC = OneVsRestClassifier(lgr,n_jobs=-1)
    OVRC.fit(X_train,y_train)
    print datetime.datetime.now()-start
    scores = OVRC.score(X_test,y_test)
    return scores, OVRC
    
lgr = LogisticRegression(verbose=1,n_jobs=-1)


score , OVRC = multiGres(lgr, X_train, y_train,X_test,y_test) 

0:01:22.786596


## Assigning coefficents

In [ ]:
def topFeatMulti(OVRC, bin_class_list, X_columns):
    coefs = []
    for x in OVRC.estimators_:
        coefs.append(x.coef_) 
    fitfeat_dict = {}
    for x,y in zip(bin_class_list, coefs):
        coef_lkup = {abs(m):(m,n) for m,n in zip(y[0],X_columns)}
        top5 = np.sort(abs(y))[0][::-1][:5]
        top5_labeld = [coef_lkup[m] for m in top5]
        print '='*60
        print x
        for m in top5_labeld:
            print m
        fitfeat_dict[x] = top5_labeld
    
    return fitfeat_dict
    

fitfeat_dict = topFeatMulti(OVRC, mlb.classes_, X.columns)

In [ ]:
def topGuesses(OVRC, X, original):
    proba = OVRC.predict_proba(X)
    print proba.shape
    df = pd.DataFrame(proba)
    df.columns = mlb.classes_
    def topPredict(y):
        top3 = y.transpose().sort_values(ascending=False)[:4]
        return [(x,z) for x,z in zip(top3.index, top3)]

    df['top_pred'] = df.apply(topPredict,axis=1)
    results = original.merge(df[['top_pred']], how='left', left_index=True, right_index=True)
    return results


predicted = topGuesses(OVRC, X, limited_postings)


In [ ]:
predicted.head(5)

## setting up multi-class SGD log regression with L1 loss

In [ ]:
sgdc = SGDClassifier(penalty='l1', loss='log',n_jobs=-1,verbose=1)
score , OVRC = multiGres(sgdc, X_train, y_train,X_test,y_test) 

In [ ]:
print score